In [1]:
import pandas as pd
import re
import pymorphy2
import joblib as pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [3]:
df = pd.read_csv('lenta-ru-news.csv', low_memory=False)

In [4]:
df.shape

(800975, 6)

In [5]:
df.head(5)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [6]:
df.tags.value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
Инновации              1
69-я параллель         1
Name: tags, Length: 94, dtype: int64

In [7]:
df.head(5)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [8]:
# возьмем из датасета только те данные, где отмечены тэги 'Россия', 'Мир', 'Экономика', 'Спорт', 'Культура' 
df = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [9]:
df.shape

(494804, 6)

In [10]:
# проверим на пустые значения
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [11]:
# посмотрим эти пустые значения (интересно же)
df[df.text.isna()].values

array([['https://lenta.ru/news/2006/04/20/student1/',
        'Прокуратура не\xa0нашла национальной почвы в\xa0нападении на\xa0индийского студента',
        nan, 'Россия', 'Все', '2006/04/20'],
       ['https://lenta.ru/news/2006/05/16/dagestan/',
        'При штурме дома в\xa0Кизилюрте ранены шестеро', nan, 'Россия',
        'Все', '2006/05/16'],
       ['https://lenta.ru/news/2006/05/24/fire/',
        'В Стамбуле горит багажное отделение аэропорта "Ататюрк"', nan,
        'Мир', 'Все', '2006/05/24'],
       ['https://lenta.ru/news/2006/05/31/an70/',
        'РФ вышла из\xa0российско-украинского проекта Ан-70', nan,
        'Россия', 'Все', '2006/05/31'],
       ['https://lenta.ru/news/2016/02/23/aquarium/',
        'Гребенщиков заработал 10\xa0тысяч рублей в\xa0омском подземном переходе',
        nan, 'Культура', 'Музыка', '2016/02/23']], dtype=object)

In [12]:
# удаляем строки с пустым значением текста
df.dropna(subset=['text'],inplace = True)

In [13]:
# снова проверим на пустые значения, чтобы убедиться, что таких нет
df.isna().sum()

url          0
title        0
text         0
topic        0
tags     15328
date         0
dtype: int64

In [14]:
# посчитаем количество статей по каждой теме
df.topic.value_counts()

Россия       160442
Мир          136620
Экономика     79528
Спорт         64413
Культура      53796
Name: topic, dtype: int64

In [15]:
# посчитаем процентное соотношение каждой темы
df.topic.value_counts(normalize=True)

Россия       0.324257
Мир          0.276112
Экономика    0.160728
Спорт        0.130180
Культура     0.108723
Name: topic, dtype: float64

In [16]:
df.shape

(494799, 6)

In [17]:
RANDOM_STATE = 42

### Базовая обработка

In [19]:
text_transformer = TfidfVectorizer()

In [20]:
%%time
text = text_transformer.fit_transform(df['text'])

CPU times: total: 2min 17s
Wall time: 2min 20s


In [21]:
df['text']

5         Как стало известно агентству Ассошиэйтед Пресс...
6         В зале игровых автоматов в третьем ярусе подзе...
7         Япония приняла решение разморозить кредиты Рос...
8         Британцы отмечают сегодня скорбную дату - втор...
9         В понедельник директор департамента внешних св...
                                ...                        
739168    Президент России Владимир Путин, выступая на з...
739169    Президент России Владимир Путин организовал дл...
739171    Протесты движения «желтых жилетов» в Париже сн...
739174    Швейцарский горнолыжник Марк Гизин неудачно пр...
739175    Президент России Владимир Путин рассказал, что...
Name: text, Length: 494799, dtype: object

In [22]:
text.shape

(494799, 821733)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [24]:
X_train.shape

(395839, 821733)

In [25]:
X_test.shape

(98960, 821733)

In [26]:
%%time
clf = tree.DecisionTreeClassifier()

CPU times: total: 0 ns
Wall time: 0 ns


In [27]:
%%time
clf.fit(X_train,y_train)

CPU times: total: 4h 55min 37s
Wall time: 5h 2min 58s


DecisionTreeClassifier()

In [28]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.7913556768523285


### 0.2 Изменим токенизатор

In [33]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [34]:
stop_russian = stopwords.words('russian')

In [35]:
stop_russian

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [36]:
%%time
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

CPU times: total: 2min 18s
Wall time: 2min 18s


In [37]:
from sklearn.ensemble import GradientBoostingClassifier

In [38]:
X_train.shape

(395839, 10000)

In [39]:
%%time
clf2 = GradientBoostingClassifier()
clf2.fit(X_train,y_train)

CPU times: total: 4h 27min 1s
Wall time: 4h 27min 33s


GradientBoostingClassifier()

In [40]:
%%time
print('TRAIN - Качество модели по метрике F1 (токенизатор настроен)',
      f1_score(y_train,clf2.predict(X_train),average='weighted'))

TRAIN - Качество модели по метрике F1 (токенизатор настроен) 0.8565170775808257
CPU times: total: 7.91 s
Wall time: 7.91 s


In [41]:
%%time
pred2 = clf2.predict(X_test)
print('TEST - Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

TEST - Качество модели по метрике F1 (токенизатор настроен) 0.8546026099506344
CPU times: total: 2.31 s
Wall time: 2.33 s


In [42]:
with open ('transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)

In [43]:
with open ('clf.pkl', 'wb') as f:
    pickle.dump(clf2, f)

### 0.3 Приведем слова к первой форме

In [44]:
pattern = r'[^А-Яа-я]+'

In [45]:
list = [re.sub(pattern, ' ', i) for i in ['проверкаyyy weyweywt iweiwue ><<++ 238238 ? 123 тест']]

In [46]:
list

['проверка тест']

In [47]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [48]:
df['text_clean'] = remove_trash(df['text'])

In [49]:
df[['text','text_clean']].head()

,text,text_clean
5,Как стало известно агентству Ассошиэйтед Пресс...,Как стало известно агентству Ассошиэйтед Пресс...
6,В зале игровых автоматов в третьем ярусе подзе...,В зале игровых автоматов в третьем ярусе подзе...
7,Япония приняла решение разморозить кредиты Рос...,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают сегодня скорбную дату - втор...,Британцы отмечают сегодня скорбную дату вторую...
9,В понедельник директор департамента внешних св...,В понедельник директор департамента внешних св...


In [50]:
df.isna().sum()

url               0
title             0
text              0
topic             0
tags          15328
date              0
text_clean        0
dtype: int64

In [ ]:
# df = df[~df.text_clean.isna()].copy()

In [ ]:
#токенизатор удалил мусор

In [52]:
df.head()

,url,title,text,topic,tags,date,text_clean
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,Как стало известно агентству Ассошиэйтед Пресс...
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,В зале игровых автоматов в третьем ярусе подзе...
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,Япония приняла решение разморозить кредиты Рос...
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31,Британцы отмечают сегодня скорбную дату вторую...
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31,В понедельник директор департамента внешних св...


In [53]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [54]:
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.32884454455872786


### 0.4 Попробуем лемматизацию

In [55]:
morph = pymorphy2.MorphAnalyzer()

In [56]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [57]:
df.text

5         Как стало известно агентству Ассошиэйтед Пресс...
6         В зале игровых автоматов в третьем ярусе подзе...
7         Япония приняла решение разморозить кредиты Рос...
8         Британцы отмечают сегодня скорбную дату - втор...
9         В понедельник директор департамента внешних св...
                                ...                        
739168    Президент России Владимир Путин, выступая на з...
739169    Президент России Владимир Путин организовал дл...
739171    Протесты движения «желтых жилетов» в Париже сн...
739174    Швейцарский горнолыжник Марк Гизин неудачно пр...
739175    Президент России Владимир Путин рассказал, что...
Name: text, Length: 494799, dtype: object

In [58]:
t = []
text = 'Минфин предлагает увеличить размер штрафа'
for word in text.split():
    if len(word)<=3:
        continue
    p = morph.parse(word)[0]
    t.append(p.normal_form)

In [59]:
" ".join(t)

'минфин предлагать увеличить размер штраф'

In [60]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

CPU times: total: 7h 20min 58s
Wall time: 7h 21min 15s


In [61]:
df.head()

,url,title,text,topic,tags,date,text_clean,text_clean_normal
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,Как стало известно агентству Ассошиэйтед Пресс...,как стать известно агентство ассошиэйтед пресс...
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,В зале игровых автоматов в третьем ярусе подзе...,зал игровой автомат третий ярус подземный комп...
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,Япония приняла решение разморозить кредиты Рос...,япония принять решение разморозить кредит росс...
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31,Британцы отмечают сегодня скорбную дату вторую...,британец отмечать сегодня скорбный дата вторую...
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31,В понедельник директор департамента внешних св...,понедельник директор департамент внешний связь...


In [ ]:
# пример чтения объекта из файла в оперативную память
with open('df_norma.pkl','rb') as f:
    df = pickle.load(f)

In [ ]:
df.head()

In [ ]:
df[['text','text_clean_normal']].tail(10)

### качество модели после лемматизации

In [62]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [63]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [ ]:
# clf_norm.predict(text_transformer.transform(['Минфин предлагает увеличить размер']))

In [64]:

print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после лемматизации 0.8155465690336047


In [ ]:


with open('data/tfidf.pkl', 'wb') as f:
    pickle.dump(clf_norm, f)

with open('data/transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)

## Сравниваем вектора текстов

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


textA = text_transformer.transform(['бывший тренер сборная россия владимир плющев']   )                              

test = cosine_similarity(text_norm, textA)

In [ ]:
testdf = pd.DataFrame(test)
testdf.columns=['vector']

In [ ]:
testdf.sort_values(by='vector',ascending=False).head(5)

In [ ]:
df[df.index==103437]['text_clean_normal'].values

### Итоги

In [67]:
print('Качество модели по метрике F1 после базовой обработки', f1_score(y_test,pred,average='weighted'))
print('Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))
print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))


Качество модели по метрике F1 после базовой обработки 0.7913556768523285
Качество модели по метрике F1 (токенизатор настроен) 0.8546026099506344
Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.32884454455872786
Качество модели по метрике F1 после лемматизации 0.8155465690336047
